In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from evaluator import ModelEvaluator
import sys, warnings

sys.path.append('../')
from datasets.setCreator import SetCreator
from datasets.setModifier import SetModifier

In [ ]:
warnings.filterwarnings("ignore")

setCreator = SetCreator()
dataset1 = setCreator.getSetList1()

In [ ]:
for currData in dataset1:
    currData['train'] = currData['train'].fillna(0)
    pca = PCA()
    pca.fit_transform(currData['train'])
    
    print(f'[SUCCESS] identified {len(pca.components_)} relevant components')
    print(pca.components_)
    print(pca.explained_variance_ratio_)

From the notes

In [ ]:
for currData in dataset1:
    trainningData = currData["train"]
    trainY = trainningData["ergot_present_in_q4"]
    trainX = setModifier.rmErgotPredictors(trainningData)

    For n in [2, 3, 4, 5]:
        poly[n] = PolynomialFeatures(n)
        X_poly[n] = poly[n].fit_transform(trainX)

        model = RandomForestClassifier(n_estimators=100,max_depth=5, n_jobs=4, random_state=2)
        rfecv = RFECV(estimator=model, n_jobs=1) # apply feature elimination/cross-validation to model
        best_feat = rfecv.fit(X_poly[2], trainY)
        X_support = X_poly[2][:, best_feat.support_] # X_support now automatically holds the best subset

        print(X_support.shape)  # tells you the best dimensions to use
        print(X_support)

In [ ]:
from sklearn.random_projection import GaussianRandomProjection

In [ ]:
for currData in dataset1:
    currData['train'] = currData['train'].fillna(0)
    gaussian_rnd_proj = GaussianRandomProjection(random_state=42)
    X_reduced = gaussian_rnd_proj.fit_transform(currData['train'])